In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import io
import numpy as np

In [ ]:
def selection_plot(dfp):
    pair = dfp["S1"].astype(str) + "-" + dfp["S2"].astype(str)
    
    fig, ax = plt.subplots(figsize=(14,6))

    # Plot
    ax.bar(pair, (dfp["P1"] / dfp["P2"]), color="steelblue")
    
    ax.axhline(1.0, color="red", linestyle="--", linewidth=1)  # reference line at 1.0
    
    ax.set_xlabel("Strategy Pair (S1-S2)")
    ax.set_ylabel("P1/P2 Ratio")
    ax.set_title("Simulation Results: P1/P2 Ratio by Strategy Pair")
    
    fig.tight_layout()
    ax.tick_params(axis='x', labelrotation=90)
    return fig, ax

def turn_card_bias_plot(df):
    # Compute difference
    LostCardDiff = df["P1 Turn Loss Average"] - df["P2 Turn Loss Average"]
    
    # Create labels
    pair = df["S1"].astype(str) + "-" + df["S2"].astype(str)
    
    # Plot
    fig, ax = plt.subplots(figsize=(14,6))
    bars = ax.bar(pair, LostCardDiff, color="steelblue")
    
    ax.axhline(0.0, color="red", linestyle="--", linewidth=1)  # reference line at 1.0
    ax.set_ylabel("Difference (P1 - P2)")
    ax.set_xlabel("Strategy Pair (S1-S2)")
    ax.set_title("Difference in Average Lost Turn Card (P1 - P2)")
    
    # Annotate values
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval,
                f"{yval:.3f}", ha="center", va="bottom", fontsize=8)
    
    ax.tick_params(axis='x', labelrotation=90)
    return fig, ax

def overlay_selection_turn_card(dfp):
    dfp = dfp.sort_values(by=["S1", "S2"])

    # Create labels
    # Compute difference
    turn_card_diff = -(dfp["P1 Turn Loss Average"] - dfp["P2 Turn Loss Average"])
    ratio_diff = (dfp["P1"] / dfp["P2"]) - 1

    # Create labels
    pair = dfp["S1"].astype(str) + "-" + dfp["S2"].astype(str)
    
    x = np.arange(len(pair))
    bar_width = 0.4

    # Compute values

    # Plot
    fig, ax = plt.subplots(figsize=(14,6))
    ax2 = ax.twinx()

    bars1 = ax.bar(x - bar_width/2, ratio_diff, width=bar_width, color="steelblue", label="P1/P2 Ratio - 1")
    bars2 = ax2.bar(x + bar_width/2, turn_card_diff, width=bar_width, color="red", label="Lost Turn Card Diff (P2 - P1)")

    ax.axhline(0.0, color="red", linestyle="--", linewidth=1)  # reference line

    # Labels and title
    ax.set_xticks(x)
    ax.set_xticklabels(pair, rotation=90)
    ax.set_xlabel("Strategy Pair (S1-S2)")
    ax.set_ylabel("P1 Residual Win/Loss Ratio")
    ax2.set_ylabel("Mean Lost Turn Card Difference (P2 - P1)")
    ax.set_title("P1/P2 Ratio vs Turn Card Difference by Strategy Pair")
    ax.legend()

    top = [max(ax.get_ylim()), max(ax2.get_ylim())]
    bottom = [min(ax.get_ylim()), min(ax2.get_ylim())]
    ax.set_ylim([min(bottom), max(top)])
    ax2.set_ylim([min(bottom), max(top)])

    plt.tight_layout()
    return fig, ax

def strategy_pair_plot(dfp):   
    # Create strategy pair labels
    pair = dfp["S1"].astype(str) + "-" + dfp["S2"].astype(str)
    # Compute % difference from 50% fairness
    P1_pct_diff = (dfp["P1"] / (dfp["P1"] + dfp["P2"]) - 0.5) * 100
    
    # Plot setup
    x = range(len(dfp))
    bar_width = 0.4
    
    fig, ax = plt.subplots(figsize=(14,6))

    bars = ax.bar(pair, P1_pct_diff, width=bar_width, label="P1 Bias (%)", alpha=0.85, color="steelblue")
    ax.axhline(0.0, color="red", linestyle="--", linewidth=1)  # reference line at 1.0
        
    
    # Add value labels above bars
    for bar in bars:
        height = bar.get_height()
        ax.annotate(
            f"{height:.2f}",
            xy=(bar.get_x() + bar.get_width() / 2, height),
            xytext=(0, 4),
            textcoords="offset points",
            ha="center",
            va="bottom",
            fontsize=8,
            rotation=45
        )
    
    # Formatting
    ax.axhline(0.0, color="red", linestyle="--", linewidth=1)
    ax.set_xticks(x)
    ax.set_xticklabels(pair, fontsize=10)
    ax.set_ylabel("Win Bias vs 50% (%)", fontsize=12)
    ax.set_xlabel("Strategy Pair (S1-S2)", fontsize=12)
    ax.set_title("P1 vs P2 Win Bias by Strategy Pair")
    ax.legend()
    ax.set_ylim(np.array(ax.get_ylim())*1.1)
    plt.xticks(rotation=90, ha="right")
    plt.tight_layout()
    return fig, ax

In [ ]:
d2vall = """S1, S2, P1, P2, Tie, P1 Turn Loss Average, P2 Turn Loss Average, Hands, Games
0, 0, 499635, 500365, 0, 8.00479, 8.00381, 283418923, 1000000
0, 1, 499485, 500515, 0, 8.00422, 8.00513, 284314502, 1000000
0, 2, 499491, 500509, 0, 8.00406, 8.00235, 283663059, 1000000
1, 0, 499283, 500717, 0, 8.00335, 8.00398, 283754309, 1000000
1, 1, 499180, 500820, 0, 8.0034, 8.00427, 284009174, 1000000
1, 2, 499947, 500053, 0, 8.00337, 8.00414, 283569165, 1000000
2, 0, 500509, 499491, 0, 8.00294, 8.00331, 283271057, 1000000
2, 1, 499856, 500144, 0, 8.00333, 8.00169, 283852523, 1000000
2, 2, 499838, 500162, 0, 8.00272, 8.00199, 283484007, 1000000
2, 3, 483174, 516826, 0, 7.9619, 7.90018, 299587274, 1000000
2, 5, 486329, 513671, 0, 7.98541, 7.97474, 297781468, 1000000
2, 7, 483960, 516040, 0, 7.97074, 7.92596, 299112511, 1000000
3, 2, 517748, 482252, 0, 7.90142, 7.96048, 299581251, 1000000
3, 3, 499952, 500048, 0, 7.86164, 7.86218, 318356897, 1000000
3, 5, 503718, 496282, 0, 7.88434, 7.93645, 315902611, 1000000
3, 7, 501054, 498946, 0, 7.87166, 7.88846, 317738623, 1000000
5, 2, 513652, 486348, 0, 7.97312, 7.98602, 297649574, 1000000
5, 3, 494810, 505190, 0, 7.93666, 7.884, 315582298, 1000000
5, 5, 499256, 500744, 0, 7.96162, 7.95976, 314636869, 1000000
5, 7, 495706, 504294, 0, 7.94459, 7.91094, 315005757, 1000000
7, 2, 516533, 483467, 0, 7.92755, 7.97009, 299211470, 1000000
7, 3, 498937, 501063, 0, 7.88978, 7.87166, 317498149, 1000000
7, 5, 503876, 496124, 0, 7.91177, 7.94305, 314467736, 1000000
7, 7, 500320, 499680, 0, 7.89648, 7.89713, 316427148, 1000000
"""

f = io.StringIO(d2vall)
df = pd.read_csv(f, delimiter=",", skipinitialspace=True)

In [ ]:
d0vall = """S1, S2, P1, P2, Tie, P1 Turn Loss Average, P2 Turn Loss Average, Hands, Games
0, 0, 500004, 499996, 0, 8.00344, 8.00395, 283494525, 1000000
1, 0, 500022, 499978, 0, 8.00299, 8.0032, 283998847, 1000000
2, 0, 500808, 499192, 0, 8.00265, 8.00306, 283286408, 1000000
3, 0, 517952, 482048, 0, 7.90354, 7.96476, 299369592, 1000000
4, 0, 479975, 520025, 0, 8.11341, 8.05339, 266695412, 1000000
5, 0, 513364, 486636, 0, 7.97427, 7.98537, 296664932, 1000000
6, 0, 483507, 516493, 0, 8.04281, 8.03097, 265584334, 1000000
7, 0, 516573, 483427, 0, 7.92807, 7.97262, 298746717, 1000000
8, 0, 481589, 518411, 0, 8.09183, 8.0475, 265951677, 1000000
0, 2, 500054, 499946, 0, 8.00443, 8.00437, 283764076, 1000000
1, 2, 499455, 500545, 0, 8.00473, 8.00441, 283784343, 1000000
2, 2, 500475, 499525, 0, 8.00352, 8.00455, 283248911, 1000000
3, 2, 516902, 483098, 0, 7.90031, 7.96093, 300379917, 1000000
4, 2, 480845, 519155, 0, 8.11655, 8.05792, 265318604, 1000000
5, 2, 514273, 485727, 0, 7.9745, 7.9862, 297600051, 1000000
6, 2, 484948, 515052, 0, 8.04357, 8.02857, 264903854, 1000000
7, 2, 517231, 482769, 0, 7.92666, 7.97076, 299037412, 1000000
8, 2, 480850, 519150, 0, 8.09419, 8.048, 265904489, 1000000"""
f = io.StringIO(d0vall)
df = pd.read_csv(f, delimiter=",", skipinitialspace=True)

In [ ]:
d = """S1, S2, P1, P2, Tie, P1 Turn Loss Average, P2 Turn Loss Average, Hands, Games
0, 0, 49867, 50133, 0, 8.00109, 7.99997, 28444920, 100000
0, 1, 50082, 49918, 0, 8.00561, 8.00535, 28306387, 100000
0, 2, 49962, 50038, 0, 8.00063, 8.00491, 28390324, 100000
0, 3, 48420, 51580, 0, 7.96556, 7.90606, 29919192, 100000
0, 4, 52087, 47913, 0, 8.0563, 8.10823, 26620616, 100000
0, 5, 48372, 51628, 0, 7.98542, 7.97617, 29728723, 100000
0, 6, 51421, 48579, 0, 8.03159, 8.04353, 26520568, 100000
0, 7, 48309, 51691, 0, 7.97606, 7.93076, 29756751, 100000
0, 8, 52052, 47948, 0, 8.05044, 8.09407, 26663275, 100000
1, 0, 49690, 50310, 0, 8.00329, 7.99963, 28374050, 100000
1, 1, 49984, 50016, 0, 8.00193, 8.00407, 28498197, 100000
1, 2, 49963, 50037, 0, 8.00044, 7.99977, 28321730, 100000
1, 3, 47693, 52307, 0, 7.96617, 7.89916, 29849804, 100000
1, 4, 51797, 48203, 0, 8.04326, 8.10621, 26840138, 100000
1, 5, 48318, 51682, 0, 7.98579, 7.97208, 29378570, 100000
1, 6, 51935, 48065, 0, 8.02501, 8.03723, 26767821, 100000
1, 7, 48141, 51859, 0, 7.97591, 7.92949, 29739674, 100000
1, 8, 52044, 47956, 0, 8.04234, 8.091, 26761540, 100000
2, 0, 50304, 49696, 0, 8.00466, 8.00909, 28338017, 100000
2, 1, 50118, 49882, 0, 7.9999, 8.00341, 28366913, 100000
2, 2, 50082, 49918, 0, 8.00673, 8.00509, 28382332, 100000
2, 3, 48243, 51757, 0, 7.95916, 7.90041, 30169512, 100000
2, 4, 51870, 48130, 0, 8.05664, 8.11545, 26584317, 100000
2, 5, 48613, 51387, 0, 7.98657, 7.9706, 29795102, 100000
2, 6, 51629, 48371, 0, 8.02787, 8.04453, 26511521, 100000
2, 7, 48223, 51777, 0, 7.97197, 7.92311, 29910449, 100000
2, 8, 52196, 47804, 0, 8.05147, 8.09352, 26572758, 100000
3, 0, 51780, 48220, 0, 7.90226, 7.96572, 29931220, 100000
3, 1, 51808, 48192, 0, 7.90431, 7.96954, 29934310, 100000
3, 2, 51870, 48130, 0, 7.89944, 7.96428, 29935030, 100000
3, 3, 49957, 50043, 0, 7.86384, 7.86808, 31761935, 100000
3, 4, 53742, 46258, 0, 7.95307, 8.0762, 27842569, 100000
3, 5, 50361, 49639, 0, 7.88589, 7.93652, 31445577, 100000
3, 6, 53398, 46602, 0, 7.93, 8.00741, 27831833, 100000
3, 7, 49974, 50026, 0, 7.86664, 7.8914, 31701461, 100000
3, 8, 53570, 46430, 0, 7.9465, 8.05702, 28045224, 100000
4, 0, 48067, 51933, 0, 8.11663, 8.05491, 26640254, 100000
4, 1, 48059, 51941, 0, 8.10959, 8.04575, 26830364, 100000
4, 2, 48041, 51959, 0, 8.11545, 8.0589, 26600390, 100000
4, 3, 46556, 53444, 0, 8.07692, 7.94875, 28042695, 100000
4, 4, 49961, 50039, 0, 8.16048, 8.15927, 25229869, 100000
4, 5, 46393, 53607, 0, 8.09877, 8.02696, 27622187, 100000
4, 6, 49550, 50450, 0, 8.14019, 8.08767, 25154688, 100000
4, 7, 46318, 53682, 0, 8.08757, 7.97856, 27910007, 100000
4, 8, 49786, 50214, 0, 8.15006, 8.13782, 25139605, 100000
5, 0, 51394, 48606, 0, 7.97679, 7.98972, 29760910, 100000
5, 1, 51473, 48527, 0, 7.97374, 7.98862, 29716184, 100000
5, 2, 51349, 48651, 0, 7.97765, 7.98477, 29764117, 100000
5, 3, 49346, 50654, 0, 7.93627, 7.88617, 31594946, 100000
5, 4, 53621, 46379, 0, 8.0261, 8.1012, 27588998, 100000
5, 5, 49998, 50002, 0, 7.95683, 7.96181, 31394865, 100000
5, 6, 53333, 46667, 0, 7.99875, 8.02687, 27577342, 100000
5, 7, 49731, 50269, 0, 7.94311, 7.90903, 31454296, 100000
5, 8, 53434, 46566, 0, 8.01816, 8.07957, 27544830, 100000
6, 0, 48548, 51452, 0, 8.04772, 8.03609, 26564007, 100000
6, 1, 48391, 51609, 0, 8.03751, 8.02686, 26679552, 100000
6, 2, 48518, 51482, 0, 8.04722, 8.03341, 26588704, 100000
6, 3, 46637, 53363, 0, 8.00326, 7.92492, 27948796, 100000
6, 4, 49937, 50063, 0, 8.08901, 8.13766, 25171799, 100000
6, 5, 46998, 53002, 0, 8.02822, 7.99694, 27559215, 100000
6, 6, 50030, 49970, 0, 8.06739, 8.06882, 25206137, 100000
6, 7, 46726, 53274, 0, 8.01572, 7.95325, 27884862, 100000
6, 8, 49987, 50013, 0, 8.08099, 8.11373, 25188068, 100000
7, 0, 51515, 48485, 0, 7.93212, 7.96844, 29788863, 100000
7, 1, 51663, 48337, 0, 7.92646, 7.97459, 29850829, 100000
7, 2, 51453, 48547, 0, 7.929, 7.97234, 29888962, 100000
7, 3, 49686, 50314, 0, 7.89002, 7.86588, 31826111, 100000
7, 4, 53608, 46392, 0, 7.97583, 8.08741, 27878300, 100000
7, 5, 50254, 49746, 0, 7.91432, 7.94397, 31540836, 100000
7, 6, 53042, 46958, 0, 7.95447, 8.01522, 27971068, 100000
7, 7, 50249, 49751, 0, 7.8952, 7.89449, 31691953, 100000
7, 8, 53657, 46343, 0, 7.9697, 8.06278, 27734990, 100000
8, 0, 48308, 51692, 0, 8.09156, 8.05187, 26637305, 100000
8, 1, 48312, 51688, 0, 8.08939, 8.03909, 26819193, 100000
8, 2, 47930, 52070, 0, 8.0909, 8.05253, 26600826, 100000
8, 3, 46508, 53492, 0, 8.05494, 7.94509, 27945946, 100000
8, 4, 50038, 49962, 0, 8.13906, 8.15063, 25171604, 100000
8, 5, 46739, 53261, 0, 8.07951, 8.01655, 27710291, 100000
8, 6, 49791, 50209, 0, 8.12117, 8.08093, 25294126, 100000
8, 7, 46509, 53491, 0, 8.06427, 7.96578, 27804877, 100000
8, 8, 49952, 50048, 0, 8.13187, 8.13539, 25319738, 100000"""
f = io.StringIO(d)
df = pd.read_csv(f, delimiter=",", skipinitialspace=True)

In [ ]:
#fig, ax = plt.subplots(figsize=(14,6))
#df.sort_values(by=["P2"], inplace=True)
#ax.bar(df["S1"], df["P2"], color="steelblue")

fig, ax = selection_plot(df.sort_values(by=["P2", "S1"]))
plt.savefig("/home/simon/war-simulation-strategy-comparison-sorted.svg")
plt.show()

turn_card_bias_plot(df)
plt.tight_layout()
plt.savefig("difference_in_average_lost_turn_card.png")
plt.show()

fig, ax = overlay_selection_turn_card(df)
ax.grid()
fig.savefig("combined-wins-and-turn-residue.svg")
plt.show()


df.sort_values(by=["S2", "S1"], inplace=True)
dfp = df[(~df["S1"].astype(int).isin([4, 6, 8]))&(~df["S2"].astype(int).isin([4, 6, 8]))]
fig, ax = strategy_pair_plot(dfp)
ax.grid()
#fig.savefig("combined-wins-and-turn-residue.svg")
plt.show()

In [ ]:
df.sort_values(by=["S1", "S2"], inplace=True)
dfp = df[(~df["S1"].astype(int).isin([4, 6, 8]))&(~df["S2"].astype(int).isin([4, 6, 8]))]
fig, ax = strategy_pair_plot(dfp)
ax.grid()
fig.savefig("win-basis-card-counting.svg")
plt.show()